In [ ]:
'''
Transformer from scratch
Based on:
    1. https://e2eml.school/transformers.html
    2. http://nlp.seas.harvard.edu/2018/04/03/attention.html
    3. https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb

'''

'\nTransformer from scratch\nBased on:\n    1. https://e2eml.school/transformers.html\n    2. http://nlp.seas.harvard.edu/2018/04/03/attention.html\n    3. https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb\n\nAuthor: Ramakrishna A\nDate: 06 March 2022\n'

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import numpy as np
import random
import math
import time

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
def tokenize_data(text):
    return [tok for tok in text.split()]

In [ ]:
SRC = Field(tokenize=tokenize_data,
           init_token='<sos>',
           eos_token='<eos>',
           lower=True,
           batch_first=True)

TRG = Field(tokenize=tokenize_data,
           init_token='<sos>',
           eos_token='<eos>',
           lower=True,
           batch_first=True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

downloading training.tar.gz


100%|██████████| 1.21M/1.21M [00:02<00:00, 603kB/s] 


downloading validation.tar.gz


100%|██████████| 46.3k/46.3k [00:00<00:00, 178kB/s]


downloading mmt_task1_test2016.tar.gz


100%|██████████| 66.2k/66.2k [00:00<00:00, 163kB/s]


In [ ]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
(train_data, valid_data, test_data),
batch_size=BATCH_SIZE,
device=device)

In [ ]:
'''
ENCODER

Encoder will have following:
    1. Positional embeddings
    2. Token embeddings
    3. Embedding = Positional embeddings + (d_model ** 1/2) * token embeddings
    4. Source mask is same shape as source sentence:
            0 if the token is '<pad>'
            1 else
'''

"\nENCODER\n\nEncoder will have following:\n    1. Positional embeddings\n    2. Token embeddings\n    3. Embedding = Positional embeddings + (d_model ** 1/2) * token embeddings\n    4. Source mask is same shape as source sentence:\n            0 if the token is '<pad>'\n            1 else\n"

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_len=100):
        super().__init__()
        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_len, hid_dim)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        self.dropout = nn.Dropout(dropout)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim, dropout, device) for _ in range(n_layers)])
    
    def forward(self, src, src_mask):
        # src: [batch_size, src_len]
        # src_mask: [batch_size, 1, 1, src_len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # pos: [batch_size, src_len]
        
        src = self.dropout(self.pos_embedding(pos) + (self.tok_embedding(src) * self.scale))
        # src: [batch_size, src_len, emb_dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
        
        # src: [batch_size, seq_len, hid_dim]
        
        return src

In [ ]:
'''
ENCODER LAYER

In the Encoder Layer we do following:
    1. Pass the src and src_mask through multi-head attention layer and then perform dropout on it
    2. Apply residual connection and pass through layer normalization layer
    3. Pass it through position-wise feed-forward network and apply dropout
    4. Apply residual connection and layer normalization to get the output

Parameters are not shared between the layers!
'''

'\nENCODER LAYER\n\nIn the Encoder Layer we do following:\n    1. Pass the src and src_mask through multi-head attention layer and then perform dropout on it\n    2. Apply residual connection and pass through layer normalization layer\n    3. Pass it through position-wise feed-forward network and apply dropout\n    4. Apply residual connection and layer normalization to get the output\n\nParameters are not shared between the layers!\n'

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src, src_mask):
        # src: [batch_size, src_len, hid_dim]
        # src_mask: [batch_size, 1, 1, src_len]
        
        # self_attention
        _src, _ = self.self_attention(src, src, src, src_mask) # (query, key, value, src_mask)
        
        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        # src: [batch_size, src_len, hid_dim]
        
        # positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        # dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        # src: [batch_size, src_len, hid_dim]
        
        return src

In [ ]:
'''
Multi-Head Attention

We calculate multi-head attention as follows:
    1. We calculate Q.W_Q, K.W_K and V.W_V with linear layers fc_q, fc_k, fc_v to get Q, K and V
    2. We split the Q, K and V of hid_dim size into n_heads using .view
    3. Calculate energy (the unnormalized attention) i.e. Q.K^t / sqrt(head_dim) where head_dim = hid_dim // n_heads
    4. Mask energy and apply softmax and dropout
    5. Multiply with V
    6. Combine n_heads together
    7. Multiply the result with W_O

K and V dimensions are always same dimensionality

Why the dropout is directly applied on attention, is not explained  proerly even it may makes the sum < 1

Notes on torch.Tensor.view():
    Returns a new tensor with the same data as self with different shape
    -1 means 'infer from other dimensions'
'''

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0, "otherwise we can't divide into multiple heads of same dimensionality"
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.dropout = nn.Dropout(dropout)
        
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        
        # query: [batch_size, query_len, hid_dim]
        # key: [batch_size, key_len, hid_dim]
        # value: [batch_size, value_len, hid_dim]
        
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        # Q: [batch_size, query_len, hid_dim]
        # K: [batch_size, key_len, hid_dim]
        # V: [batch_size, value_len, hid_dim]
        
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    
        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]
        # i.e. we just splitted the last dimension which is hid_dim into n_heads * head_dim
        
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        # energy: [batch_size, n_heads, query_len, key_len]
        
        if mask is not None:
            energy = energy.masked_fill(mask==0, -1e10)
        
        attention = torch.softmax(energy, dim=-1)
        # attention: [batch_size, n_heads, query_len, key_len]
        
        scaled_dot_product_attention = torch.matmul(self.dropout(attention), V)
        # scaled_dot_product_attention: [batch_size, n_heads, query_len, head_dim]
        
        scaled_dot_product_attention = scaled_dot_product_attention.permute(0, 2, 1, 3).contiguous() # to make the permuation inplace
        # scaled_dot_product_attention: [batch_size, query_len, n_heads, head_dim]
        
        scaled_dot_product_attention = scaled_dot_product_attention.view(batch_size, -1, self.hid_dim)
        # scaled_dot_product_attention: [batch_size, query_len, hid_dim]
        
        scaled_dot_product_attention = self.fc_o(scaled_dot_product_attention)
        # scaled_dot_product_attention: [batch_size, query_len, hid_dim]
        
        return scaled_dot_product_attention, attention

In [ ]:
'''
Position-wise Feedforward layer

A two-layer feedforward network with ReLU activation
'''
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x: [batch_size, seq_len, hid_dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        # x: [batch_size, seq_len, pf_dim]
        
        x = self.fc_2(x)
        # x: [batch_size, seq_len, hid_dim]
        
        return x


In [ ]:
'''
DECODER:

We do following:
    1. Same as Encoder
    2. Masked-MultiHead Self Attention on decoder inputs
    3. Cross Attention between Encoder output and Decoder self-attention
    4. Add and Norm
    5. FC_Out and Softmax
'''

class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_len=100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_len, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, n_heads, pf_dim, dropout, device) for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
    
    def forward(self, trg, enc_src, trg_mask, src_mask):
        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hid_dim]
        # trg_mask: [batch_size, 1, trg_len, trg_len]
        # src_mask: [batch_size, 1, 1, src_len]
        
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # pos: [batch_size, trg_len]
        
        trg = self.dropout(self.pos_embedding(pos) + (self.tok_embedding(trg) * self.scale))
        # trg: [batch_size, trg_len, hid_dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        # trg: [batch_size, trg_len, hid_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)
        # output: [batch_size, trg_len, output_dim]

        return output, attention

In [ ]:
'''
DecoderLayer

We do following:
    1. Masked Multi-Head Attention: (Q, K and V are trg)
    2. Residual connection and LayerNorm
    3. Cross attention: (Q is output of Masked Multi-Head attention and K, V are output of Encoder)
    4. Residual connection and LayerNorm
    5. Positionwise Feedforward
    6. Residual connection and LayerNorm
    7. Get output
'''

class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)

        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.enc_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
    
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, trg, enc_src, trg_mask, src_mask):
        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hid_dim]
        # trg_mask: [batch_size, 1, trg_len, trg_len]
        # src_mask: [batch_size, 1, 1, src_len]

        # masked multi-head attention and add and norm
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # cross attention and add and norm
        _trg, attention = self.enc_attention(trg, enc_src, enc_src, src_mask)
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # positionwise feedforward and add and norm
        _trg = self.positionwise_feedforward(trg)
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hid_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

In [ ]:
'''
Seq2Seq

1. Source mask is 0 if <pad> else 1. Since we want a 4-d tensor for multi-head attention, we will unsqueeze it as follows:
    [batch_size, n_heads, src_len, src_len]
2. Target mask is sequential mask and square subsequent mask. Square mask is created by torch.tril which is a square matrix
    Here, each token (row) is allowed to look at column
'''

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        # src: [batch_size, src_len]
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask
    
    def make_trg_mask(self, trg):
        # trg: [batch_size, trg_len]
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=self.device)).bool()
        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask
    
    def forward(self, src, trg):
        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_mask]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)
        # enc_src: [batch_size, src_len, hid_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

In [ ]:
# params
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [ ]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,413,848 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (tok_embedding): Embedding(9597, 256)
    (pos_embedding): Embedding(100, 256)
    (dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attention): MultiHeadAttentionLayer(
          (dropout): Dropout(p=0.1, inplace=False)
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256

In [ ]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
# train
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
# eval
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
# time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# finally training
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 16s
	Train Loss: 4.237 | Train PPL:  69.175
	 Val. Loss: 3.127 |  Val. PPL:  22.802
Epoch: 02 | Time: 0m 16s
	Train Loss: 2.929 | Train PPL:  18.716
	 Val. Loss: 2.440 |  Val. PPL:  11.478
Epoch: 03 | Time: 0m 17s
	Train Loss: 2.354 | Train PPL:  10.529
	 Val. Loss: 2.127 |  Val. PPL:   8.391
Epoch: 04 | Time: 0m 16s
	Train Loss: 1.991 | Train PPL:   7.323
	 Val. Loss: 1.981 |  Val. PPL:   7.249
Epoch: 05 | Time: 0m 16s
	Train Loss: 1.721 | Train PPL:   5.589
	 Val. Loss: 1.908 |  Val. PPL:   6.740
Epoch: 06 | Time: 0m 16s
	Train Loss: 1.511 | Train PPL:   4.532
	 Val. Loss: 1.869 |  Val. PPL:   6.482
Epoch: 07 | Time: 0m 16s
	Train Loss: 1.338 | Train PPL:   3.811
	 Val. Loss: 1.857 |  Val. PPL:   6.403
Epoch: 08 | Time: 0m 16s
	Train Loss: 1.193 | Train PPL:   3.297
	 Val. Loss: 1.842 |  Val. PPL:   6.306
Epoch: 09 | Time: 0m 17s
	Train Loss: 1.068 | Train PPL:   2.911
	 Val. Loss: 1.887 |  Val. PPL:   6.598
Epoch: 10 | Time: 0m 16s
	Train Loss: 0.961 | Train PPL